## Deep Learning Approach 

There are 2 significant parts of data.
- Shop details
- Past foot traffic

As part of Deep Learning approach, we will:
- Fit a model for foot traffic
- Cluster data to generate extra features to overcome issues of Sparse data
- Fit a model to improve accuracy by utilizing Shop details

### Reshaping Data
In order to create a model in machine learning, we need to have inputs and ouputs. However, if we are trying to machine learn daily foot traffic, we will have to divide it in inputs/ouputs. Assuming, we will feed 15 days of data to predict next 15 days, we need to have a 2 matrix of [nx15]. 


In [9]:
import pandas as pd
import numpy as np
import sqlite3
import os.path
from sqlalchemy import create_engine
from matplotlib import pyplot as plt
pd.set_option("display.max_rows",15)
%matplotlib inline

In [10]:
conn = create_engine('sqlite:///../dataset/ijcai_cff_2017.db')

In [11]:
train_data_raw = pd.read_sql_query("select shop_id, \
(total_view + total_pay) as total_today, date(ts) as ts from train_data where ts > '2016-07-01'", 
                               con = conn)
train_data_raw

,shop_id,total_today,ts
0,1,268.0,2016-07-01
1,1,218.0,2016-07-02
2,1,224.0,2016-07-03
3,1,260.0,2016-07-04
4,1,267.0,2016-07-05
5,1,243.0,2016-07-06
6,1,272.0,2016-07-07
...,...,...,...
152204,2000,195.0,2016-10-25
152205,2000,197.0,2016-10-26


In [12]:
train_data = pd.read_sql_query("select shop_id, \
(total_view + total_pay) as input_foot_traffic from train_data where ts > '2016-06-01'", 
                               con = conn)

time_split_period = 15
time_periods_total = int(train_data.shape[0] / time_split_period) + 1
time_day_arr = np.tile(np.arange(0,15), time_periods_total)
time_period_arr = [np.full(shape=time_split_period, fill_value=i ) for i in range(time_periods_total)]
time_period_arr = np.concatenate(time_period_arr)
print (time_period_arr[0])

train_data["day"] = time_day_arr[0:train_data.shape[0]]
train_data["period"] = time_period_arr[0:train_data.shape[0]]
train_data['output_foot_traffic'] = train_data.input_foot_traffic.shift(-1)

train_data

0


,shop_id,input_foot_traffic,day,period,output_foot_traffic
0,1,NaN,0,0,NaN
1,1,NaN,1,0,NaN
2,1,NaN,2,0,NaN
3,1,NaN,3,0,NaN
4,1,NaN,4,0,NaN
5,1,NaN,5,0,NaN
6,1,NaN,6,0,NaN
...,...,...,...,...,...
184895,2000,195.0,5,12326,197.0
184896,2000,197.0,6,12326,148.0


In [13]:
train_data_p = train_data.pivot_table(index = ['shop_id', 'period'],
                       columns = 'day',
                       values = ['input_foot_traffic','output_foot_traffic']
                      )

In [14]:
train_data_p

input_foot_traffic                                            \
day                            0      1      2      3      4      5      6    
shop_id period                                                                
1       0                     NaN    NaN    NaN    NaN    NaN    NaN    NaN   
        1                     NaN    NaN    NaN    NaN    NaN    NaN  178.0   
        2                   268.0  218.0  224.0  260.0  267.0  243.0  272.0   
        3                   288.0  233.0  259.0  266.0  248.0  262.0    NaN   
        4                   225.0  264.0  233.0  424.0  419.0  539.0  267.0   
        5                   257.0  272.0  267.0  310.0  261.0  253.0  266.0   
        6                   304.0  320.0  419.0  397.0  393.0  349.0  393.0   
...                           ...    ...    ...    ...    ...    ...    ...   
2000    12320                 NaN   42.0   45.0    NaN  146.0  174.0  170.0   
        12321                43.0   39.0  190.0  131.0  185.0  207.0  163.0   
        12322                76.0  177.0  195.0  187.0  168.0  171.0   98.0   
        12323                 NaN    NaN    NaN  176.0  179.0  136.0  123.0   
        12324               170.0  160.0  187.0  142.0   53.0   70.0  162.0   
        12325                38.0   80.0   79.0  174.0  200.0  196.0  187.0   
        12326               200.0  151.0   72.0   63.0  181.0  195.0  197.0   

                                     ...   output_foot_traffic                \
day                7      8      9   ...                    5      6      7    
shop_id period                       ...                                       
1       0         NaN    NaN    NaN  ...                   NaN    NaN    NaN   
        1       233.0  250.0  234.0  ...                 178.0  233.0  250.0   
        2       320.0  262.0  275.0  ...                 272.0  320.0  262.0   
        3       270.0  256.0    NaN  ...                   NaN  270.0  256.0   
        4       282.0  302.0  343.0  ...                 267.0  282.0  302.0   
        5       281.0  272.0  288.0  ...                 266.0  281.0  272.0   
        6       364.0  317.0  383.0  ...                 393.0  364.0  317.0   
...               ...    ...    ...  ...                   ...    ...    ...   
2000    12320   111.0   47.0   35.0  ...                 170.0  111.0   47.0   
        12321    80.0   63.0  188.0  ...                 163.0   80.0   63.0   
        12322   130.0  259.0  212.0  ...                  98.0  130.0  259.0   
        12323   149.0  125.0  135.0  ...                 123.0  149.0  125.0   
        12324   182.0  138.0  127.0  ...                 162.0  182.0  138.0   
        12325   192.0  212.0  162.0  ...                 187.0  192.0  212.0   
        12326   148.0  145.0   78.0  ...                 197.0  148.0  145.0   

                                                                 
day                8      9      10     11     12     13     14  
shop_id period                                                   
1       0         NaN    NaN    NaN    NaN    NaN    NaN    NaN  
        1       234.0  230.0  237.0  191.0  254.0  261.0  268.0  
        2       275.0  297.0  243.0  220.0  237.0  265.0  288.0  
        3         NaN  225.0  281.0  236.0  290.0  280.0  225.0  
        4       343.0  265.0  284.0  273.0  294.0  285.0  257.0  
        5       288.0  353.0  322.0  332.0  302.0  296.0  304.0  
        6       383.0  364.0  337.0  296.0  312.0  339.0  337.0  
...               ...    ...    ...    ...    ...    ...    ...  
2000    12320    35.0  124.0  134.0  153.0  102.0  143.0   43.0  
        12321   188.0  216.0  169.0  184.0  161.0   53.0   76.0  
        12322   212.0  210.0  170.0  156.0   88.0    NaN    NaN  
        12323   135.0   31.0   51.0   52.0  162.0  166.0  170.0  
        12324   127.0  159.0   43.0   43.0   53.0   37.0   38.0  
        12325   162.0   87.0   64.0  179.0  175.0  178.0  200.0  
        12326    78.0   70.0 

In [15]:
train = train_data_p.dropna(how='any').reset_index().drop('period', axis=1)

/home/ritesh_malaiya/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel/__main__.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  if __name__ == '__main__':


In [16]:
train.to_pickle("cff_15dayperiod.pkl")
train

shop_id input_foot_traffic                                            \
day                           0      1      2      3      4      5      6   
0          1              268.0  218.0  224.0  260.0  267.0  243.0  272.0   
1          1              225.0  264.0  233.0  424.0  419.0  539.0  267.0   
2          1              257.0  272.0  267.0  310.0  261.0  253.0  266.0   
3          1              304.0  320.0  419.0  397.0  393.0  349.0  393.0   
4          1              337.0  238.0  352.0  339.0  356.0  230.0  254.0   
5          1              206.0  336.0  349.0  255.0  327.0  312.0  291.0   
6          1              279.0  294.0  284.0  232.0  267.0  206.0  231.0   
...      ...                ...    ...    ...    ...    ...    ...    ...   
5306    1999              404.0  392.0  535.0  540.0  326.0  320.0  410.0   
5307    1999              331.0  501.0  442.0  304.0  284.0  318.0  361.0   
5308    1999              358.0  329.0  352.0  298.0  333.0  332.0  380.0   
5309    2000              149.0  185.0  129.0   76.0   61.0  186.0  160.0   
5310    2000               43.0   39.0  190.0  131.0  185.0  207.0  163.0   
5311    2000              170.0  160.0  187.0  142.0   53.0   70.0  162.0   
5312    2000               38.0   80.0   79.0  174.0  200.0  196.0  187.0   

                    ...   output_foot_traffic                              \
day       7      8  ...                     5      6      7      8      9   
0     320.0  262.0  ...                 272.0  320.0  262.0  275.0  297.0   
1     282.0  302.0  ...                 267.0  282.0  302.0  343.0  265.0   
2     281.0  272.0  ...                 266.0  281.0  272.0  288.0  353.0   
3     364.0  317.0  ...                 393.0  364.0  317.0  383.0  364.0   
4     226.0  242.0  ...                 254.0  226.0  242.0  272.0  297.0   
5     275.0  294.0  ...                 291.0  275.0  294.0  283.0  291.0   
6     250.0  229.0  ...                 231.0  250.0  229.0  268.0  235.0   
...     ...    ...  ...                   ...    ...    ...    ...    ...   
5306  520.0  414.0  ...                 410.0  520.0  414.0  363.0  241.0   
5307  387.0  439.0  ...                 361.0  387.0  439.0  399.0  363.0   
5308  532.0  565.0  ...                 380.0  532.0  565.0  330.0  301.0   
5309  191.0  161.0  ...                 160.0  191.0  161.0  160.0   43.0   
5310   80.0   63.0  ...                 163.0   80.0   63.0  188.0  216.0   
5311  182.0  138.0  ...                 162.0  182.0  138.0  127.0  159.0   
5312  192.0  212.0  ...                 187.0  192.0  212.0  162.0   87.0   

                                         
day      10     11     12     13     14  
0     243.0  220.0  237.0  265.0  288.0  
1     284.0  273.0  294.0  285.0  257.0  
2     322.0  332.0  302.0  296.0  304.0  
3     337.0  296.0  312.0  339.0  337.0  
4     258.0  250.0  254.0  194.0  206.0  
5     270.0  299.0  281.0  254.0  279.0  
6     276.0  195.0  250.0  278.0  283.0  
...     ...    ...    ...    ...    ...  
5306  291.0  296.0  313.0  277.0  331.0  
5307  354.0  325.0  384.0  415.0  358.0  
5308  307.0  303.0  364.0  477.0  458.0  
5309   59.0  170.0  111.0  133.0  144.0  
5310  169.0  184.0  161.0   53.0   76.0  
5311   43.0   43.0   53.0   37.0   38.0  
5312   64.0  179.0  175.0  178.0  200.0  

[5313 rows x 31 columns]